In [3]:
#START
import pandas as pd
from sqlalchemy import create_engine

In [4]:
## Importing the Netflix CSV and converting it into a dataframe to clean and edit ##
netflix_csv = "./Resources/netflix_titles.csv"
netflix_df = pd.read_csv(netflix_csv)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [5]:
## Importing the Movie Streaming CSV and converting it into a dataframe to clean and edit ##
movies_csv = "./Resources/MoviesOnStreaming.csv"
movies_df = pd.read_csv(movies_csv)
movies_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [6]:
netflix_col = ["show_id", "title", "rating", "duration"]
netflix_fil = netflix_df.loc[netflix_df["type"] == "Movie"]
netflix_fix = netflix_fil.reindex(columns=netflix_col)
netflix_fix.dropna()
netflix_fix.head()

,show_id,title,rating,duration
0,81145628,Norm of the North: King Sized Adventure,TV-PG,90 min
1,80117401,Jandino: Whatever it Takes,TV-MA,94 min
4,80125979,#realityhigh,TV-14,99 min
6,70304989,Automata,R,110 min
7,80164077,Fabrizio Copano: Solo pienso en mi,TV-MA,60 min


In [7]:
movies_fixed = movies_df.drop(columns=['Hulu','Prime Video', 'Disney+','Type','Directors','Genres','Country', 'Language','Runtime'])
movies_fixed = movies_fixed.rename(columns={"Rotten Tomatoes": "rotten_tomato",
                                           "ID": "id",
                                           "Title": "title",
                                           "Year": "movie_year",
                                           "Age": "age",
                                           "IMDb": "imdb",
                                           "Netflix": "netflix"})

movies_fixed.set_index("id")
movies_fixed = movies_fixed.drop(movies_fixed.filter(regex="Unname"),axis=1)

movies_fixed.head()

,id,title,movie_year,age,imdb,rotten_tomato,netflix
0,1,Inception,2010,13+,8.8,87%,1
1,2,The Matrix,1999,18+,8.7,87%,1
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1
3,4,Back to the Future,1985,7+,8.5,96%,1
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1


In [31]:
#Connection to the database
connection_string = "postgres:postgres@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['netflix_data', 'streaming_data']

In [14]:
#load data into netflex data table
netflix_fix.to_sql(name='netflix_data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "netflix_data_pkey"
DETAIL:  Key (show_id)=(81145628) already exists.

[SQL: INSERT INTO netflix_data (show_id, title, rating, duration) VALUES (%(show_id)s, %(title)s, %(rating)s, %(duration)s)]
[parameters: ({'show_id': 81145628, 'title': 'Norm of the North: King Sized Adventure', 'rating': 'TV-PG', 'duration': '90 min'}, {'show_id': 80117401, 'title': 'Jandino: Whatever it Takes', 'rating': 'TV-MA', 'duration': '94 min'}, {'show_id': 80125979, 'title': '#realityhigh', 'rating': 'TV-14', 'duration': '99 min'}, {'show_id': 70304989, 'title': 'Automata', 'rating': 'R', 'duration': '110 min'}, {'show_id': 80164077, 'title': 'Fabrizio Copano: Solo pienso en mi', 'rating': 'TV-MA', 'duration': '60 min'}, {'show_id': 70304990, 'title': 'Good People', 'rating': 'R', 'duration': '90 min'}, {'show_id': 80169755, 'title': 'Joaquín Reyes: Una y no más', 'rating': 'TV-MA', 'duration': '78 min'}, {'sh

In [15]:
#load data into streaming data table
movies_fixed.to_sql(name='streaming_data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "streaming_data_pkey"
DETAIL:  Key (id)=(1) already exists.

[SQL: INSERT INTO streaming_data (id, title, movie_year, age, imdb, rotten_tomato, netflix) VALUES (%(id)s, %(title)s, %(movie_year)s, %(age)s, %(imdb)s, %(rotten_tomato)s, %(netflix)s)]
[parameters: ({'id': 1, 'title': 'Inception', 'movie_year': 2010, 'age': '13+', 'imdb': 8.8, 'rotten_tomato': '87%', 'netflix': 1}, {'id': 2, 'title': 'The Matrix', 'movie_year': 1999, 'age': '18+', 'imdb': 8.7, 'rotten_tomato': '87%', 'netflix': 1}, {'id': 3, 'title': 'Avengers: Infinity War', 'movie_year': 2018, 'age': '13+', 'imdb': 8.5, 'rotten_tomato': '84%', 'netflix': 1}, {'id': 4, 'title': 'Back to the Future', 'movie_year': 1985, 'age': '7+', 'imdb': 8.5, 'rotten_tomato': '96%', 'netflix': 1}, {'id': 5, 'title': 'The Good, the Bad and the Ugly', 'movie_year': 1966, 'age': '18+', 'imdb': 8.8, 'rotten_tomato': '97%', 'netflix': 1}, {'id': 6

In [33]:
#Test pulling query from Netfix database
pd.read_sql_query('select * from netflix_data', con=engine).head()

,show_id,title,duration,rating
0,81145628,Norm of the North: King Sized Adventure,90 min,TV-PG
1,80117401,Jandino: Whatever it Takes,94 min,TV-MA
2,80125979,#realityhigh,99 min,TV-14
3,70304989,Automata,110 min,R
4,80164077,Fabrizio Copano: Solo pienso en mi,60 min,TV-MA


In [34]:
#Test pulling query from Steaming database
pd.read_sql_query('select * from streaming_data', con=engine).head()

,id,title,movie_year,age,imdb,rotten_tomato,netflix
0,1,Inception,2010,13+,8.8,87%,1
1,2,The Matrix,1999,18+,8.7,87%,1
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1
3,4,Back to the Future,1985,7+,8.5,96%,1
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1


In [35]:
#Join the data Neflix db and Streaming db
pd.read_sql_query('select * from netflix_data b left join streaming_data a on a.title = b.title  ', con=engine).head()

,show_id,title,duration,rating,id,title,movie_year,age,imdb,rotten_tomato,netflix
0,70131314,Inception,148 min,PG-13,1.0,Inception,2010,13+,8.8,87%,1.0
1,20557937,The Matrix,136 min,R,2.0,The Matrix,1999,18+,8.7,87%,1.0
2,80219127,Avengers: Infinity War,150 min,PG-13,3.0,Avengers: Infinity War,2018,13+,8.5,84%,1.0
3,81002747,Spider-Man: Into the Spider-Verse,117 min,PG,6.0,Spider-Man: Into the Spider-Verse,2018,7+,8.4,97%,1.0
4,70108777,Inglourious Basterds,153 min,R,10.0,Inglourious Basterds,2009,18+,8.3,89%,1.0


In [36]:
#Join the data Neflix db and Streaming db to have a clearner view
pd.read_sql_query('select b.title, a.movie_year, a.age, a.imdb, a.rotten_tomato, b.duration,\
                  b.rating from netflix_data b left join streaming_data a on a.title = b.title  ', con=engine).head()

,title,movie_year,age,imdb,rotten_tomato,duration,rating
0,Inception,2010,13+,8.8,87%,148 min,PG-13
1,The Matrix,1999,18+,8.7,87%,136 min,R
2,Avengers: Infinity War,2018,13+,8.5,84%,150 min,PG-13
3,Spider-Man: Into the Spider-Verse,2018,7+,8.4,97%,117 min,PG
4,Inglourious Basterds,2009,18+,8.3,89%,153 min,R


In [56]:
#Join tables and highlight the null in the age group
pd.read_sql_query('select b.title, a.movie_year, a.age, a.imdb, a.rotten_tomato, b.duration,\
                  b.rating from netflix_data b left join streaming_data a on a.title = b.title where age is not null  ', con=engine).head()

,title,movie_year,age,imdb,rotten_tomato,duration,rating
0,Inception,2010,13+,8.8,87%,148 min,PG-13
1,The Matrix,1999,18+,8.7,87%,136 min,R
2,Avengers: Infinity War,2018,13+,8.5,84%,150 min,PG-13
3,Spider-Man: Into the Spider-Verse,2018,7+,8.4,97%,117 min,PG
4,Inglourious Basterds,2009,18+,8.3,89%,153 min,R
